Algo genetique avec:
- un crossover sur la 1 et 2 parties suelement
- Crossover uniform 
- reduire l'espace de recherche
- Encodage Statique


#1. Importer DataSet


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp "/content/drive/MyDrive/Colab Notebooks/Projet Master/DataSets/BaseDeDonnees_2.zip" minutiae_detection.zip 

In [3]:
from zipfile import ZipFile

print("Deecompression .....")
with ZipFile('minutiae_detection.zip') as zip:
    zip.extractall('Dataset')

Deecompression .....


#2. Loads libs

In [4]:
import numpy as np
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Conv2D, Dense, Input, MaxPool2D, Dropout, Activation, Normalization, Flatten, Reshape
from keras.activations import relu, softmax
from keras.optimizers import SGD, Adam
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import random 
from itertools import combinations
import pandas as pd
import datetime
import csv
import cv2 as cv
import os
import time

#3. Model Creation

In [5]:

def CreateModel1(optimizer=None,input_shape=(),nb_classe=2, individual=[],
                    version="static",loss='categorical_crossentropy',
                    mertics="accuracy"):
    """
        Fonction de ceation du pemiere modele utilisé pour la classification des blocs
            -  Input Shape : dimension d'entré
            -  nb_classe: nombre de classe de classification dans notre cas c'est 2
            -  individual: le chromosome de l'individu
            -  version: version de l'encodage soit "dynamique" ou "statique"
            -  loss: la fonction erreur
            -  metrics: metrique d'evaluation
    """
    my_model = Sequential(name="Model")
    my_model.add(Input(shape=input_shape))
    #my_model.add(Reshape((28,28,1)))

    conv_layers = individual[0]
    dense_layers = individual[1]

    if version=="dynamic":
        for i,layer in enumerate(conv_layers):
            my_model.add(Conv2D(layer[0], layer[1], name=f'conv{i}',padding="same")) #Add conv parameters
            if layer[3] == 1: my_model.add(Activation(relu))
            if layer[2] != 0: my_model.add(MaxPool2D((layer[2],layer[2]),padding="same"))

        my_model.add(Flatten())

        for i,layer in enumerate(dense_layers):
            my_model.add(Dense(layer[0]))
            if layer[1] == 1:  my_model.add(Activation(relu))

    elif version=="static":
        for i,layer in enumerate(conv_layers):
            if layer != []:
                my_model.add(Conv2D(layer[0], layer[1], name=f'conv{i}',padding="same")) #Add conv parameters
                if layer[3] == 1:  my_model.add(Activation(relu))
                if layer[2] != 0: my_model.add(MaxPool2D((layer[2],layer[2]),padding="same"))
    
        my_model.add(Flatten())

        for i,layer in enumerate(dense_layers):
            if layer != []:
                my_model.add(Dense(layer[0]))
                if layer[1] == 1:  my_model.add(Activation(relu))
    else:
        print("Erreur de version dans le parametre version")
    
    my_model.add(Dense(nb_classe,activation=softmax))

    my_model.compile(loss=loss, optimizer=optimizer, metrics=[mertics])

    return my_model


def LossFunction(y_true, y_pred):
    return np.mean(np.square(y_true - y_pred))
    
def CreateModel2(optimizer=None,input_shape=(),nb_features=2, individual=[],
                version="statique",mertics="accuracy"):
    """
        Fonction de ceation du 2eme modele utilisé detecter la minuties dnas les blocs
            -  Input Shape : dimension d'entré
            -  nb_feature: nombre de parametre predire
            -  individual: le chromosome de l'individu
            -  version: version de l'encodage soit "dynamique" ou "statique"
            -  loss: la fonction erreur
            -  metrics: metrique d'evaluation
    """
    my_model = Sequential(name="Model")
    my_model.add(Input(shape=input_shape))
    #my_model.add(Reshape((28,28,1)))

    conv_layers = individual[0]
    dense_layers = individual[1]

    if version=="dynamique":
        for i,layer in enumerate(conv_layers):
            my_model.add(Conv2D(layer[0], layer[1], name=f'conv{i}',padding="same")) #Add conv parameters
            if layer[3] == 1: my_model.add(Activation(relu))
            if layer[2] != 0: my_model.add(MaxPool2D((layer[2],layer[2]),padding="same"))

        my_model.add(Flatten())

        for i,layer in enumerate(dense_layers):
            my_model.add(Dense(layer[0]))
            if layer[1] == 1:  my_model.add(Activation(relu))

    elif version=="statique":
        for i,layer in enumerate(conv_layers):
            if layer != []:
                my_model.add(Conv2D(layer[0], layer[1], name=f'conv{i}',padding="same")) #Add conv parameters
                if layer[3] == 1:  my_model.add(Activation(relu))
                if layer[2] != 0: my_model.add(MaxPool2D((layer[2],layer[2]),padding="same"))
    
        my_model.add(Flatten())

        for i,layer in enumerate(dense_layers):
            if layer != []:
                my_model.add(Dense(layer[0]))
                if layer[1] == 1:  my_model.add(Activation(relu))
    else:
        print("Erreur de version dans le parametre version")
    
    my_model.add(Dense(nb_features))
    my_model.compile(loss=LossFunction, optimizer=optimizer, metrics=[mertics])

    return my_model


#4. Genetic Algorithme

## Search space

In [6]:
nb_bloc_conv_valeus = [5,6,7,8]
nb_filters_valeus = [32,64,128,256] #[10,20,30,40]
filter_size_valeus = [3,5,7,9]
nb_dense_layer_valeus = [1,2,3]
nb_dense_units_valeus = [64,128,256,512]
polling_size_valeus = [0,3,5] #Dans le cas ou c'est 0 on ajoute pas de pooling

learning_rate = [0.1,0.01,0.02,0.001,0.002,0.0001] 

search_space_conv = [nb_filters_valeus,filter_size_valeus,polling_size_valeus,[0,1]]
search_space_dense = [nb_dense_units_valeus,[0,1]]


## Population generation

In [7]:
def InitPopulation(population_size,version="static"):

    population = []
    individual = []
    conv_layers =  []
    layer = []
    dense_layers = []

    for i in range(0,population_size):
    
        conv_layers =  []
        nb_conv_layers = random.choice(nb_bloc_conv_valeus)
        for j in range(nb_conv_layers):
            layer.append(random.choice(nb_filters_valeus))
            layer.append(random.choice(filter_size_valeus))
            layer.append(random.choice(polling_size_valeus))
            layer.append(random.choice([0,1])) #Choixe d'ajouter ou pas une couche d'activation
            conv_layers.append(layer)
            layer = []
        
        if version == "static":
            for i in range(nb_conv_layers, max(nb_bloc_conv_valeus)):
                conv_layers.append([])
        
        individual.append(conv_layers)
        
        dense_layers = []
        nb_dense_layers = random.choice(nb_dense_layer_valeus)
        for j in range(nb_dense_layers):
            layer.append(random.choice(nb_dense_units_valeus))
            layer.append(random.choice([0,1])) #Choix d'ajouter ou pas une couche d'activation
            dense_layers.append(layer)
            layer=[]
        
        if version =="static":
            for i in range(nb_dense_layers, max(nb_dense_layer_valeus)):
                dense_layers.append([])
        individual.append(dense_layers)

        population.append(individual)
        individual = []

    return population

## Evaluation

In [8]:
def Fitness(individual,optimizer=None,input_shape=(),
                     nb_classe=2,train_set = [],
                     test_set=[],nb_epochs = 4, 
                     batch_size = 100,validation_split = 0.2):
    #try:
    if len(train_set) == 1:
        model = CreateModel1(optimizer=optimizer,input_shape=input_shape,nb_classe=nb_classe,individual=individual)
        model.fit(x = train_set[0], batch_size=batch_size, epochs=nb_epochs)
        # validation_split=validation_split,
        test_loss, test_acc = model.evaluate(test_set[0], steps=len(test_set[0]))
        #print(f"test loss:{test_loss}, test accuracy:{test_acc}")
    else:
        model = CreateModel2(optimizer=optimizer,input_shape=input_shape,nb_features=nb_classe,individual=individual)
        model.fit(x = train_set[0],y = train_set[1],batch_size=batch_size, epochs=nb_epochs)
        # validation_split=validation_split,
        test_loss, test_acc = model.evaluate(test_set[0],test_set[1], steps=len(test_set[0]))

    #except:
    #    print("Oups!")
    #    return 0,0
    
    return test_loss,test_acc

def EvaluatePopulation(population = [], optimizer = None,input_shape=(),
                        train_set = [], test_set=[], nb_epochs = 15,
                        batch_size = 50,file_path1=None,file_path2=None):

    evaluation = []
    if len(train_set) != 0: 
        
        for i,individual in enumerate(population):
            print("Evaluation individu: ",i)
            debut = time.time()
            test_loss, fitness = Fitness(optimizer=optimizer, individual = individual,input_shape=input_shape,
                                    train_set=train_set,test_set=test_set,nb_epochs=nb_epochs,batch_size=batch_size)
            #evaluated_population[tuple(individual)] = fitness
            fin = time.time()
            
            data = {"test accuracy":fitness,"time":fin-debut}
            WriteOnCSV(file_path2,data)

            evaluation.append((individual,fitness))

            with open(file_path1,"a") as f:
                f.write(f"{individual}\n")
            f.close()

    return evaluation

def WriteOnCSV(file_path, data):
    file = open(file_path, "a",newline='')
    writer = csv.DictWriter(file, fieldnames=list(data.keys()))
    writer.writerow(data)
    file.close()


## Selection parents

In [9]:


def SelectBestSolution(population = []):
    sorted_population = sorted(population, key=lambda x: x[1])
    return sorted_population[-1]


def BestRankedSelection(nb_parents, population:list):

    population_sorted = sorted(population, key=lambda x: x[1],reverse=True)
    parents  = [parent[0] for parent in population_sorted][0:nb_parents]
    
    return parents

def rank_selection(population, num_parents):
    fitness = [individual.fitness for individual in population]
    rank = np.argsort(np.argsort(fitness))[::-1]
    selected_parents = []
    for i in range(num_parents):
        rand = np.random.randint(0, sum(range(len(rank)))+1)
        for j in range(len(rank)):
            rand -= j
            if rand < 0:
                selected_parents.append(population[rank[j]])
                break
    return selected_parents

def random_selection(population, nb_parents):
    selected = []
    parents = []
    for i in range(nb_parents):
        r = random.randint(0,len(population))
        while r in selected:
            r = random.randint(0,len(population))
    
        selected.append(r)
        parents.append(list(population.keys())[r])
  
    return parents


def SelectionAphaBeta(population,children, alpha, beta):
    
    pass

## Corossover

In [10]:

def CrossoverConv(parent_1, parent_2, proba_crossover = 0.7):
    """
        Crossover applique sur la partie des couches convolution
            -  proba_crossover: la probabilité de faire un crossover
    """
    if random.random() < proba_crossover:
        child_1,child_2 = [],[]
        min_length = min(len(parent_1[0]),len(parent_2[0]))
        random_point = random.randint(0,min_length)
        child_1.append(parent_1[0][0:random_point]+parent_2[0][random_point:len(parent_2[0])])
        child_2.append(parent_2[0][0:random_point]+parent_1[0][random_point:len(parent_1[0])])
        min_length = min(len(parent_1[1]),len(parent_2[1]))
        random_point = random.randint(0,min_length)
        child_1.append(parent_1[1])
        child_2.append(parent_2[1])

        return child_1,child_2
    else:
        return parent_1, parent_2

def CrossoverDense(parent_1, parent_2, proba_crossover=0.7):
    """
        Crossover applique sur la partie des couches fully connected
            -  proba_crossover: la probabilité de faire un crossover
    """
    if random.random() < proba_crossover:
        child_1,child_2 = [],[]
        min_length = min(len(parent_1[0]),len(parent_2[0]))
        random_point = random.randint(0,min_length)
        child_1.append(parent_1[1][0:random_point]+parent_2[1][random_point:len(parent_2[0])])
        child_2.append(parent_2[1][0:random_point]+parent_1[1][random_point:len(parent_1[0])])
        min_length = min(len(parent_1[1]),len(parent_2[1]))
        random_point = random.randint(0,min_length)
        child_1.append(parent_1[0])
        child_2.append(parent_2[0])

        return child_1,child_2
    else:
        return parent_1, parent_2

def Crossover2Parties(parent_1, parent_2, proba_crossover):
    """
        Crossover applique sur les deux parties du chromosome
            -  proba_crossover: la probabilité de faire un crossover
    """
    if random.random() < proba_crossover:
        child_1,child_2 = [],[]
        min_length = min(len(parent_1[0]),len(parent_2[0]))
        random_point = random.randint(0,min_length)
        child_1.append(parent_1[0][0:random_point]+parent_2[0][random_point:len(parent_2[0])])
        child_2.append(parent_2[0][0:random_point]+parent_1[0][random_point:len(parent_1[0])])
        min_length = min(len(parent_1[1]),len(parent_2[1]))
        random_point = random.randint(0,min_length)
        child_1.append(parent_1[1][0:random_point]+parent_2[1][random_point:len(parent_2[1])])
        child_2.append(parent_2[1][0:random_point]+parent_1[1][random_point:len(parent_1[1])])

        return child_1,child_2
    else:
        return parent_1, parent_2


def UniformCrossover(parent_1,parent_2,proba_crossover):
    """
        Crossover uniforme applique sur les deux parties du chromosome 
            -  proba_crossover: la probabilité de faire un crossover
    """
    child_1, child_2 = [[],[]],[[],[]]

    for i in range(len(parent_1[0])):
        if random.random() < proba_crossover: 
            child_1[0][i] = parent_1[0][i]
            child_2[0][i] = parent_2[0][i]
        else : 
            child_1[0][i] = parent_2[0][i]
            child_2[0][i] = parent_1[0][i]

    for i in range(len(parent_1[1])):
        if random.random() < proba_crossover: 
            child_1[1][i] = parent_1[1][i]
            child_2[1][i] = parent_2[1][i]
        else : 
            child_1[1][i] = parent_2[1][i]
            child_2[1][i] = parent_1[1][i]

    return child_1,child_2


## Mutation

In [11]:

def AddConvLayer(individual,i):
    
    layer = []
    layer.append(random.choice(nb_filters_valeus))
    layer.append(random.choice(filter_size_valeus))
    layer.append(random.choice(polling_size_valeus))
    layer.append(random.choice([0,1]))

    individual[0][i] = layer
    
    return individual
  

def DelConvLayer(individual,i):

    individual[0][i] = []
    return individual
 

def AlterConvLayer(individual,i):

    rand_param = random.randrange(0,len(individual[0][i]))
    val_selected = random.choice(search_space_conv[rand_param])
    while val_selected == individual[0][i][rand_param]:
        val_selected = random.choice(search_space_conv[rand_param])
    individual[0][i][rand_param] = val_selected
    return individual

def AddDenseLayer(individual,i):

    layer = []
    
    layer.append(random.choice(nb_dense_units_valeus))
    layer.append(random.choice([0,1]))

    individual[1][i] = layer
    
    return individual
  

def DelDenseLayer(individual,i):
    individual[1][i] = []
    return individual
  

def AlterDenseLayer(individual,i):

    rand_param = random.randrange(0,len(individual[1][i]))
    val_selected = random.choice(search_space_dense[rand_param])
    while val_selected == individual[1][i][rand_param]:
        val_selected = random.choice(search_space_dense[rand_param])
    individual[1][i][rand_param] = val_selected

    return individual

def Mutation(individual, proba_mutation=1):
    count1, count2 = 0,0
    del_layer_conv, del_layer_dense = True,True

    for layer in individual[0]: 
        if layer == []: count1 += 1
    if count1<= min(nb_bloc_conv_valeus): del_layer_conv = False

    for layer in individual[1]:
            if layer == []:count2 += 1
    if count2<= min(nb_bloc_conv_valeus): del_layer_dense = False
        

    for i, layer in enumerate(individual[0]):
        if random.random() < proba_mutation:

            if layer != []: 
                r = random.choice([1,2])
                if r == 1 : individual = AlterConvLayer(individual, i)
                elif del_layer_conv : individual = DelConvLayer(individual,i)
                else : individual = AlterConvLayer(individual, i)

            else : individual = AddConvLayer(individual,i)
        
    for i, layer in enumerate(individual[1]):
        if random.random() < proba_mutation:

            if layer != []: 
                r = random.choice([1,2])
                if r == 1 : individual = AlterDenseLayer(individual, i)
                elif del_layer_dense : individual = DelDenseLayer(individual,i)
                else : individual = AlterDenseLayer(individual, i)

            else: individual = AddDenseLayer(individual,i)
    
    return individual



## select next generation

In [12]:
def SelectNextGeneration(population = [],children=[],generation_size=10, elite_frac = 0.5, children_frac = 0.5):
  
    nb_population = int(generation_size*elite_frac)
    nb_children = int(generation_size*children_frac)

    population_sorted = sorted(population, key=lambda x: x[1], reverse=True)  

    next_generation = []
    next_generation +=  [indiv[0] for indiv in population_sorted[0:nb_population]]
    next_generation += random.sample(children,nb_children)

    if (nb_population+nb_children) < generation_size :
        nb_random_indivs = int((1 - (elite_frac+children_frac)) * generation_size)

        for i in range(nb_random_indivs):
            random_number = random.sample(list(population.keys()),nb_random_indivs)

    return next_generation

## Algorithme

In [13]:

def GeneticAlgorithme(population_size, nb_generation,nb_parents,elite_frac,
                      children_frac,optimizer,input_shape, train_set, test_set, 
                      nb_epochs,batch_size, file_path1,file_path2, proba_crossover,proba_mutation ):
    best_in_generation = []
    #initalise la population aleatoirement
    population = InitPopulation(population_size)
    
    population_evaluated = EvaluatePopulation(population=population,optimizer=optimizer,input_shape=input_shape, train_set=train_set,test_set= test_set, 
                                        nb_epochs=nb_epochs,batch_size=batch_size, file_path1=file_path1,file_path2=file_path2)
    best_in_generation.append(SelectBestSolution(population_evaluated))

    for i in range(nb_generation):
    
        with open(file_path1, "a") as f:
            f.write(f"Generation:{i} \n")
        f.close()

        print("Debut generation: ",i)
        #Evaluer la population
        """
        population_evaluated = EvaluatePopulation(population=population,optimizer=optimizer,input_shape=input_shape, train_set=train_set,test_set= test_set, 
                                        nb_epochs=nb_epochs,batch_size=batch_size, file_path1=file_path1,file_path2=file_path2)
        """
        
        parents = BestRankedSelection(nb_parents,population_evaluated)
        
        #Appliquer le crossover pour cree de nouveau enfants
        new_children = []
        for parent_1, parent_2 in combinations(parents,2):
            #appliquer le crossover sur chaque 2 pers de parents
            child_1,child_2 = CrossoverConv(parent_1,parent_2,proba_crossover)
            new_children.append(child_1)
            new_children.append(child_2)

        #Appliquer la mutation sur les enfants
        children_after_mutation = []
        for child in new_children:
            mutated_child = Mutation(child,proba_mutation)
            children_after_mutation.append(mutated_child)
        
        #selection la population de la future generation
        population = SelectNextGeneration(population_evaluated, children_after_mutation,population_size,elite_frac,children_frac)
        
        
        #Evaluer la population
        population_evaluated = EvaluatePopulation(population=population,optimizer=optimizer,input_shape=input_shape, train_set=train_set,test_set= test_set, 
                                        nb_epochs=nb_epochs,batch_size=batch_size, file_path1=file_path1,file_path2=file_path2)
        best_in_generation.append(SelectBestSolution(population_evaluated))
    
    #Selectionner la meilleur solution attiente
    #best_in_generation = SelectBestSolution(population_evaluated, best_in_generation)

    return best_in_generation


#4. Datas

In [14]:


def LoadDataBase1(TrainingPath:str, TestPath:str,batchsize=150):

    train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True,
        validation_split=0.2)

    training_set= train_datagen.flow_from_directory(
        TrainingPath,
        target_size=(32, 32),
        color_mode="grayscale",
        batch_size=batchsize,
        class_mode='categorical'
        )
    
    
    test_datagen = ImageDataGenerator(
            rescale=1./255)

    test_set= test_datagen.flow_from_directory(
            TestPath,
            target_size=(32, 32),
            color_mode="grayscale",
            batch_size=batchsize,
            class_mode='categorical'
            )
    
    return training_set,test_set

def LoadDataBase2(DataSet_Path:str,Images_Path:str, TestSplit=0.2):
    
    TrainSet_X, TestSet_X= [],[]
    df = pd.read_csv(DataSet_Path)
    Train, Test = train_test_split(df, test_size=TestSplit,shuffle=True)


    for path in Train["Images"]:
      img = cv.imread(Images_Path+"/"+path, cv.IMREAD_GRAYSCALE)
      #img = np.reshape(img,(32,32,1))

      TrainSet_X.append(img)

    for path in Test["Images"]:
      img = cv.imread(Images_Path+"/"+path, cv.IMREAD_GRAYSCALE)
      #img = np.reshape(img,(32,32,1))

      TestSet_X.append(img)

    TrainSet_Y = Train[["X","Y"]]
    TestSet_Y = Test[["X","Y"]]

    TrainSet_X = np.array(TrainSet_X)
    TestSet_X = np.array(TestSet_X)
    print(f"Taille de Train {len(TrainSet_X)} {len(TrainSet_Y)}")
    print(f"Taille de Test {len(TestSet_X)} {len(TestSet_Y)}")

    return TrainSet_X,TestSet_X,TrainSet_Y, TestSet_Y

# Test

In [ ]:
NB_OF_GENERATION = 10
POPULATION_SIZE = 20
NB_PARENTS = 15
INPUT_SHAPE =(32,32,1)
BATCH_SIZE = 150
NB_EPOCHS = 15
ELITE_FRAC = 0.5
CHILDREN_FRAC = 0.5
TEST_SIZE = 0.4
PROBA_MUTATION = 0.7
PROBA_CROSSOVER = 0.7


date = datetime.datetime.now()
date = date.strftime("%m_%d_%H_%M_%S")

FILE_NAME1 = f"file_{date}.txt"
FILE_NAME2 = f"file_{date}.csv"

f = open(FILE_NAME1,"w")
f.write(f"NB_OF_GENERATION = {NB_OF_GENERATION}\nPOPULATION_SIZE = {POPULATION_SIZE}\nINPUT_SHAPE "+
        f"= {INPUT_SHAPE}\nBATCH_SIZE = {BATCH_SIZE}\nNB_EPOCHS = {NB_EPOCHS}\nELITE_FRAC = {ELITE_FRAC}\nCHILDREN_FRAC = "+
        f"{CHILDREN_FRAC}\nTEST_SIZE = {TEST_SIZE}\nPROBA_MUTATION = {PROBA_MUTATION}\nPROBA_CROSSOVER = {PROBA_CROSSOVER}")

f.close()

if not os.path.isfile(FILE_NAME2):
    columns = ["test accuracy", "time"]
    csv_file = open(FILE_NAME2, 'w',newline='')
    writer = csv.DictWriter(csv_file,fieldnames=columns)
    writer.writeheader()
    csv_file.close()

csv_file = "/content/Dataset/BaseDeDonnees_2/Labels.csv"
image_path = "/content/Dataset/BaseDeDonnees_2/Images" 

TrainSet_X,TestSet_X,TrainSet_Y, TestSet_Y = LoadDataBase2(csv_file,image_path)


optimizer = SGD(0.02)
best_solution = GeneticAlgorithme(POPULATION_SIZE,NB_OF_GENERATION,NB_PARENTS,ELITE_FRAC,CHILDREN_FRAC,optimizer,
                INPUT_SHAPE,[TrainSet_X,TrainSet_Y],[TestSet_X,TestSet_Y],NB_EPOCHS,BATCH_SIZE, FILE_NAME1,FILE_NAME2,PROBA_CROSSOVER,PROBA_MUTATION)
print("Best Solution: ",best_solution)


Taille de Train 62412 62412
Taille de Test 26748 26748
Evaluation individu:  0
Epoch 1/15
417/417 [==============================] - 10s 21ms/step - loss: 249.0287 - accuracy: 0.5224
Epoch 2/15
417/417 [==============================] - 8s 20ms/step - loss: 249.5252 - accuracy: 0.5224
Epoch 3/15
417/417 [==============================] - 9s 20ms/step - loss: 249.5253 - accuracy: 0.5224
Epoch 4/15
417/417 [==============================] - 8s 20ms/step - loss: 249.5252 - accuracy: 0.5224
Epoch 5/15
417/417 [==============================] - 8s 20ms/step - loss: 249.5254 - accuracy: 0.5224
Epoch 6/15
417/417 [==============================] - 8s 20ms/step - loss: 249.5253 - accuracy: 0.5224
Epoch 7/15
417/417 [==============================] - 8s 19ms/step - loss: 249.5253 - accuracy: 0.5224
Epoch 8/15
417/417 [==============================] - 8s 20ms/step - loss: 249.5251 - accuracy: 0.5224
Epoch 9/15
417/417 [==============================] - 8s 20ms/step - loss: 249.5253 - accuracy: 

In [ ]:
!cp -r "file_{date}.txt" "/content/drive/MyDrive/Colab Notebooks/Projet Master/Tests/file_{date}.txt"